# anchor-fluvial-sanity
## Fluvial Preview Sanity Check
Code anchor: `analog_image_generator.preview.generate_preview`

This notebook renders a lightweight preview for the fluvial pipeline so we can visually sanity check
each implementation increment without waiting for full UX/reporting wiring. It pairs with the Task Master
implementation loop described in Section 8 of `docs/CODEX_RUNBOOK.md`.

### Workflow Notes
- Run `scripts/render_preview.py --env fluvial` to reproduce the assets saved here.
- Update Task Master before/after each run so previews map back to task IDs.
- When generators mature, swap placeholder metadata for actual masks + QC flags.

In [ ]:
from __future__ import annotations

from pathlib import Path
import json
from typing import Any

import matplotlib.pyplot as plt
import pandas as pd

from analog_image_generator.preview import ENVIRONMENTS, generate_preview

plt.rcParams['figure.figsize'] = (6, 6)


In [ ]:
ENV: str = 'fluvial'
PREVIEW_DIR = Path('dist/previews')
PREVIEW_DIR.mkdir(parents=True, exist_ok=True)

assert ENV in ENVIRONMENTS, f'Unknown environment: {ENV}'

def render_preview(*, seed: int = 0, width: int = 512, height: int = 512, params: dict[str, Any] | None = None):
    analog, masks, metadata = generate_preview(
        ENV,
        seed=seed,
        width=width,
        height=height,
        params=params,
    )
    fig, axes = plt.subplots(1, max(1, len(masks) + 1), squeeze=False)
    axes[0, 0].imshow(analog, cmap='gray', vmin=0.0, vmax=1.0)
    axes[0, 0].set_title('Analog')
    axes[0, 0].axis('off')
    for ax, (name, array) in zip(axes[0, 1:], masks.items()):
        ax.imshow(array, cmap='gray', vmin=0.0, vmax=1.0)
        ax.set_title(name)
        ax.axis('off')
    fig.suptitle(f"{ENV.title()} preview - source: {metadata['source']}")
    fig.tight_layout()
    return metadata


In [ ]:
preview_metadata = render_preview(seed=42)
preview_metadata


In [ ]:
def list_saved_previews(dir_path: Path) -> pd.DataFrame:
    records = []
    for json_path in sorted(dir_path.glob('*.json')):
        payload = json.loads(json_path.read_text())
        records.append({
            'env': payload.get('env'),
            'seed': payload.get('seed'),
            'source': payload.get('source'),
            'analog': payload.get('artifacts', {}).get('analog'),
        })
    return pd.DataFrame.from_records(records)

list_saved_previews(PREVIEW_DIR)


In [ ]:
def task_snapshot(env_tag: str = 'fluvial-v1') -> pd.DataFrame:
    data = json.loads(Path('.taskmaster/tasks/tasks.json').read_text())
    tasks = data.get(env_tag, {}).get('tasks', [])
    rows = []
    for task in tasks:
        rows.append({
            'id': task['id'],
            'title': task['title'],
            'status': task['status'],
        })
    return pd.DataFrame(rows)

task_snapshot()
